![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)
This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA

In [ ]:
import SDDP, JuMP, ASDDiP, PyPlot

In [ ]:
import LaTeXStrings: @L_str

# A toy model for ALD-SDDiP

## Description

$$\begin{array}{rl}
      \min  & \mathbb{E}\left[\sum\limits_t \beta^{t-1}|x_t|\right] \\
\text{s.t.} & \quad x_t = x_{t-1} + c_t + \xi_t \\
            & \quad c_t \in \{\pm 1\}
\end{array} $$

## The noise $\xi_t$

Is identically distributed ("periodic system"), and **symmetric** (important for the analytic solution below).

In [ ]:
srand(11111)
A_noise   = 0.4
num_noise = 5
noise = randn(num_noise)
noise = A_noise * [noise; -noise];

# Analytic solution using symmetry

In [ ]:
cost(x) = abs.(x)
next_step(x) = (x .>=0 ).*(x-1) + (x .< 0).*(x+1)

In [ ]:
function Q2_bar(x2, nmax, noise, beta, t)
    nsamples = length(noise)
    return sum([solve_bin_sym(x2+xi, nmax, noise, beta, t+1) for xi in noise])/nsamples
end
    
function solve_bin_sym(x, nmax, noise, beta=0.5, t=1)
    if t > nmax
        return 0
    end
    
    x2 = next_step(x)
    return cost(x2) + beta*Q2_bar(x2, nmax, noise, beta, t)
end

## Very detailed (small steps) calculations

In [ ]:
xs = linspace(-3,3,1000)
Qt1000 = Vector{Float64}[]
for t = 1:7
    print(t, " -> ")
    @time v = Q2_bar(xs, 8, noise, discount, t)
    push!(Qt1000, v)
end

In [ ]:
beta = 0.5
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
for t = 1:8
    ax1[:plot](xs, Qt1000[9-t]*beta^(t-1), label="$t")
    xypos, xyneg = unwrap(xs, Qt1000[9-t], t-1, beta)
    ax2[:plot](xypos..., color="C$(t-1)", label="$t")
    ax2[:plot](xyneg..., color="C$(t-1)")
end
ax1[:set_title]("Cost-to-go")
ax2[:set_title]("Future cost (Average)")
ax2[:legend](bbox_to_anchor=(1,0.5), loc="center left");

## Calculations for a 61-point discretization

In [ ]:
ts = -3:0.1:3
discount = 0.9
Qt3 = Vector{Float64}[]
for t = 1:7
    print(t, " -> ")
    @time v = Q2_bar(ts, 8, noise, discount, t)
    push!(Qt3, v)
end

In [ ]:
for t = 1:7
    PyPlot.plot(ts, Qt3[t], label="$t")
end
PyPlot.title(L"Future cost function $Q_t$")
PyPlot.legend(title=L"stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

# Using only Strenghtened benders cuts:

This corresponds to $\rho = 0$, for all stages and iterations.

In [ ]:
ramp_mode = :None

In [ ]:
niters = 100
include("control.jl")

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title=L"Stage $t$")
PyPlot.title(L"Future cost functions $\mathfrak{Q}_t$")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
    PyPlot.suptitle("Stage $t")
    qt = ASDDiP.Qtilde(m,t,1,ts)
    qf = ASDDiP.Qfrak(m,t,1,ts)
    ax1[:plot](ts, Qt3[t], label=L"$Q$: exact")
    ax1[:plot](ts, qt,     label=L"$\tilde{Q}$: mean of next stage approximations")
    ax1[:plot](ts, qf,     label=L"$\mathfrak{Q}$: current stage approximation")
    ax1[:legend]()
    ax1[:set_title]("Future cost functions")
    ax1[:grid]()
    ax2[:plot](ts, qt     - qf, label=L"$\tilde{Q} - \mathfrak{Q}$")
    ax2[:plot](ts, Qt3[t] - qf, label=L"$Q - \mathfrak{Q}$")
    ax2[:plot](ts, Qt3[t] - qt, label=L"$Q - \tilde{Q}$")
    ax2[:set_title]("Differences")
    ax2[:legend]()
    ax2[:grid]()
    PyPlot.savefig("/tmp/1d_sb_stage$t.pdf")
end

# Using ALD

We estimate $\displaystyle Lip_t = \beta^{t-1} + \beta^t + \ldots + \beta^{8-1} = \beta^{t-1}(1 + \ldots + \beta^{8-t}) = \frac{1 - \beta^{8+1-t}}{1 - \beta}\beta^{t-1}$.

# First strategy: homothetic

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(1, \max\left(0, \frac{n-15}{15}\right)\right) \cdot Lip_t. $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase $\rho_t$ until it reaches $Lip_t$ in 15 stages;
- and keep at $Lip_t$ until the end.

In [ ]:
ramp_mode = :simple

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    qe = Qt3[t]*discount^t
    qt = ASDDiP.Qtilde(m,t,1,ts)
    qf = ASDDiP.Qfrak(m,t,1,ts)

    fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(12,4))
    PyPlot.suptitle("Stage $t")

    ax1[:plot](ts, qe, label=L"$Q$: exact")
    ax1[:plot](ts, qt, label=L"$\tilde{Q}$: mean of next stage approximations")
    ax1[:plot](ts, qf, label=L"$\mathfrak{Q}$: current stage approximation")
    ax1[:legend]()
    ax1[:set_title]("Future cost functions")
    ax1[:grid]()

    ax2[:plot](ts, qt - qf, label=L"$\tilde{Q} - \mathfrak{Q}$")
    ax2[:plot](ts, qe - qf, label=L"$Q - \mathfrak{Q}$")
    ax2[:plot](ts, qe - qt, label=L"$Q - \tilde{Q}$")
    ax2[:set_title]("Differences")
    ax2[:legend]()
    ax2[:grid]()

    PyPlot.savefig("/tmp/1d_sb_stage$t.pdf")
end

# Second strategy: parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $Lip_t$; (so that different stages "saturate" at different times)
- and keep at $Lip_t$ until the end.

In [ ]:
ramp_mode = :parallel

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
    PyPlot.suptitle("Stage $t")
    qt = ASDDiP.Qtilde(m,t,1,ts)
    qf = ASDDiP.Qfrak(m,t,1,ts)
    ax1[:plot](ts, qt, label=L"$\tilde{Q}$: mean of next stage")
    ax1[:plot](ts, qf, label=L"$\mathfrak{Q}$: Lower bound with ALD cuts")
    ax1[:legend]()
    ax1[:set_title]("Future cost functions")
    ax1[:grid]()
    ax2[:plot](ts, qt - qf)
    ax2[:set_title](L"Difference $\tilde{Q} - \mathfrak{Q}$")
    ax2[:grid]()
end

# Third strategy: double of parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(2 Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $2 Lip_t$; (so that different stages "saturate" at different times)
- and keep at $2 Lip_t$ until the end.

In [ ]:
ramp_mode = :parallel2

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();
#PyPlot.show()

In [ ]:
ts = -3:0.1:3
for t = 1:7
    fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
    PyPlot.suptitle("Stage $t")
    qt = ASDDiP.Qtilde(m,t,1,ts)
    qf = ASDDiP.Qfrak(m,t,1,ts)
    ax1[:plot](ts, qt, label=L"$\tilde{Q}$: mean of next stage")
    ax1[:plot](ts, qf, label=L"$\mathfrak{Q}$: Lower bound with ALD cuts")
    ax1[:legend]()
    ax1[:set_title]("Future cost functions")
    ax1[:grid]()
    ax2[:plot](ts, qt - qf)
    ax2[:set_title](L"Difference $\tilde{Q} - \mathfrak{Q}$")
    ax2[:grid]()
end